In [1]:
import music21
from music21 import converter, instrument, note, chord
import tensorflow as tf
from tensorflow import keras
import numpy as np
import glob

In [2]:
def read_midis(file):
    notes = []
    midi = converter.parse(file)
    notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

In [3]:
midi_files = [file for file in glob.glob("midis/*.mid")]

In [4]:
final_notes = [read_midis(file) for file in midi_files]
all_notes = []
for note_array in final_notes:
    for elem in note_array:
        all_notes.append(elem)

In [5]:
n_vocab = len(set(all_notes))

In [17]:
sequence_length = 25
pitchnames = sorted(set(item for item in all_notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [18]:
network_in = []
network_out = []
for i in range(0, len(all_notes) - sequence_length):
    seq_in = all_notes[i:i+sequence_length]
    seq_out = all_notes[i + sequence_length]
    network_in.append([note_to_int[char] for char in seq_in])
    network_out.append(note_to_int[seq_out])

n_patterns = len(network_in)
network_in_array = np.reshape(network_in, (n_patterns, sequence_length,1))
network_out = keras.utils.to_categorical(network_out)

In [19]:
model = keras.Sequential()
model.add(keras.layers.LSTM(128, input_shape = (network_in_array.shape[1], network_in_array.shape[2]), return_sequences = True))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.LSTM(64))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(n_vocab, activation = 'softmax'))

In [20]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [21]:
history = model.fit(network_in_array, network_out, epochs = 150)

Train on 3584 samples
Epoch 1/150
3584/3584 [==============================] - 4s 1ms/sample - loss: 2.5647
Epoch 2/150
3584/3584 [==============================] - 2s 568us/sample - loss: 2.2856
Epoch 3/150
3584/3584 [==============================] - 2s 566us/sample - loss: 2.2259
Epoch 4/150
3584/3584 [==============================] - 2s 576us/sample - loss: 2.1245
Epoch 5/150
3584/3584 [==============================] - 2s 604us/sample - loss: 2.0201
Epoch 6/150
3584/3584 [==============================] - 2s 563us/sample - loss: 1.8965
Epoch 7/150
3584/3584 [==============================] - 2s 569us/sample - loss: 1.8493
Epoch 8/150
3584/3584 [==============================] - 2s 574us/sample - loss: 1.7726
Epoch 9/150
3584/3584 [==============================] - 2s 621us/sample - loss: 1.6657
Epoch 10/150
3584/3584 [==============================] - 2s 564us/sample - loss: 1.6123
Epoch 11/150
3584/3584 [==============================] - 2s 563us/sample - loss: 1.5854
Epoch 12/1

Epoch 93/150
3584/3584 [==============================] - 2s 589us/sample - loss: 0.4210
Epoch 94/150
3584/3584 [==============================] - 2s 590us/sample - loss: 0.4323
Epoch 95/150
3584/3584 [==============================] - 2s 594us/sample - loss: 0.4222
Epoch 96/150
3584/3584 [==============================] - 2s 598us/sample - loss: 0.4194
Epoch 97/150
3584/3584 [==============================] - 2s 593us/sample - loss: 0.3991
Epoch 98/150
3584/3584 [==============================] - 2s 592us/sample - loss: 0.4206
Epoch 99/150
3584/3584 [==============================] - 2s 596us/sample - loss: 0.4216
Epoch 100/150
3584/3584 [==============================] - 2s 595us/sample - loss: 0.3992
Epoch 101/150
3584/3584 [==============================] - 2s 597us/sample - loss: 0.3948
Epoch 102/150
3584/3584 [==============================] - 2s 587us/sample - loss: 0.4028
Epoch 103/150
3584/3584 [==============================] - 2s 595us/sample - loss: 0.4150
Epoch 104/150
358

In [22]:
start = np.random.randint(0, len(network_in)-1)
int_to_note = dict((number,note) for number, note in enumerate(pitchnames))

pattern = network_in[start]
pred_out = []

for note_index in range(500):
    pred_in = np.reshape(pattern, (1, len(pattern),1))
    prediction = model.predict(pred_in, verbose = 0)
    
    index = np.argmax(prediction)
    result = int_to_note[index]
    pred_out.append(result)
    
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [23]:
offset = 0
output_notes = []

for pattern in pred_out:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.ElectricGuitar()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.ElectricGuitar()
        output_notes.append(new_note)
    offset += 0.25

In [24]:
midi_stream = music21.stream.Stream(output_notes)
midi_stream.show('text')

{0.0} <music21.note.Note D>
{0.25} <music21.note.Note D>
{0.5} <music21.note.Note C#>
{0.75} <music21.note.Note C#>
{1.0} <music21.note.Note G#>
{1.25} <music21.note.Note G#>
{1.5} <music21.note.Note F#>
{1.75} <music21.note.Note A>
{2.0} <music21.note.Note A>
{2.25} <music21.note.Note F#>
{2.5} <music21.note.Note G#>
{2.75} <music21.note.Note G#>
{3.0} <music21.note.Note F#>
{3.25} <music21.note.Note G>
{3.5} <music21.note.Note G>
{3.75} <music21.note.Note G#>
{4.0} <music21.note.Note F#>
{4.25} <music21.note.Note C>
{4.5} <music21.note.Note C>
{4.75} <music21.note.Note B>
{5.0} <music21.note.Note C>
{5.25} <music21.note.Note C#>
{5.5} <music21.note.Note G#>
{5.75} <music21.note.Note G#>
{6.0} <music21.note.Note F#>
{6.25} <music21.note.Note A>
{6.5} <music21.note.Note A>
{6.75} <music21.note.Note F#>
{7.0} <music21.note.Note E->
{7.25} <music21.note.Note E->
{7.5} <music21.note.Note D>
{7.75} <music21.note.Note D>
{8.0} <music21.note.Note C#>
{8.25} <music21.note.Note C#>
{8.5} <musi

In [25]:
midi_stream.write('midi', fp = 'test_out.mid')

'test_out.mid'